In [223]:
#label encoder, deepfm
#Private score: 0.87022, Public score: 0.88019
#去掉RecordID
#Private score: 0.87777, Public score: 0.87523
#不增加特征，稳定的validatdata，去掉RecordID
#Private score: 0.86755, Public score: 0.87485
#不增加特征，稳定的validatdata（会抖动）
#Private Score: 0.87111, Public Score: 0.87904
#PrivateScore: 0.87377, Public score: 0.87752*

#targetencoder, deepFM
#Private score: 0.86666, Public score: 0.87390

#onehotencoder, deepFM
#Private score: 0.85333, Public score: 0.87295

In [224]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib # 注意这个也要import一次
import matplotlib.pyplot as plt 
import seaborn as sns
import pickle
from sklearn import metrics
from sklearn.metrics import roc_auc_score, accuracy_score, recall_score, mean_squared_error, roc_curve, auc, log_loss
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

from deepctr.models import DeepFM
from deepctr.feature_column import SparseFeat, DenseFeat, get_feature_names

import warnings
warnings.filterwarnings("ignore")

In [225]:
#加载数据
train_data = pd.read_csv('./data/train.csv')
test_data = pd.read_csv('./data/test.csv')

In [226]:
datas = pd.concat([train_data, test_data], axis=0)

In [227]:
#增加交叉特征
# datas['total_contact'] = datas['campaign'] + datas['previous']
# datas['contact_diff'] = datas['campaign'] - datas['previous']

In [228]:
# datas.drop(columns=['RecordID'], inplace=True)

In [229]:
datas['subscribe'] = datas['subscribe'].map({'no': 0, 'yes': 1})

In [230]:
sparse_features = datas.select_dtypes(include='O').columns
dense_features = datas.select_dtypes(exclude='O').columns

In [231]:
dense_features

Index(['RecordID', 'age', 'duration', 'campaign', 'pdays', 'previous',
       'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m',
       'nr.employed', 'subscribe'],
      dtype='object')

In [232]:
dense_features = dense_features.delete(dense_features.get_loc('subscribe'))

In [233]:
#处理缺失值
# data[sparse_features] = data[sparse_features].fillna('-1', )
# data[dense_features] = data[dense_features].fillna(0, )

In [234]:
target = ['subscribe']

In [235]:
# 1.Label Encoding for sparse features,and do simple Transformation for dense features
for feat in sparse_features:
    lbe = LabelEncoder()
    datas[feat] = lbe.fit_transform(datas[feat])
    
mms = MinMaxScaler(feature_range=(0, 1))
datas[dense_features] = mms.fit_transform(datas[dense_features])

In [236]:
# 2.count #unique features for each sparse field,and record dense feature field name
fixlen_feature_columns = [SparseFeat(feat, vocabulary_size=datas[feat].max() + 1, embedding_dim=4)
                          for i, feat in enumerate(sparse_features)] + [DenseFeat(feat, 1, ) for feat in dense_features]

In [237]:
#特征共用，放给低维linear_feature_columns和高维dnn_feature_columns
dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns

In [238]:
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

In [239]:
# 3.generate input data for model
# train, test = train_test_split(datas, test_size=0.2, random_state=2020)
# 数据集切分
train = datas[datas['subscribe'].notnull()]
test = datas[datas['subscribe'].isnull()]
test.drop(['subscribe'], axis=1, inplace=True)

In [240]:
#增加检验，此种方式拆分的结果比使用deepctr.fit自带的拆分更稳定
train_X,eval_X,train_y,eval_y = train_test_split(train[feature_names], train[target], test_size=0.2, random_state=2022, stratify=train[target])

In [241]:
train_model_input = {name: train_X[name] for name in feature_names}
eval_model_input = {name: eval_X[name] for name in feature_names}
test_model_input = {name: test[name] for name in feature_names}

In [242]:
# 4.Define Model,train,predict and evaluate
model = DeepFM(linear_feature_columns, dnn_feature_columns, task='binary')
model.compile("adam", "binary_crossentropy", metrics=['binary_crossentropy','accuracy'], )

In [243]:
# history = model.fit(train_model_input, train[target].values, batch_size=256, epochs=10, verbose=2, validation_split=0.2, )

history = model.fit(train_model_input, train_y.values, batch_size=256, epochs=10, verbose=2, 
                    validation_data=(eval_model_input, eval_y.values),)# validation_batch_size=50)

Epoch 1/10
71/71 - 1s - loss: 0.4310 - binary_crossentropy: 0.4310 - accuracy: 0.8014 - val_loss: 0.3277 - val_binary_crossentropy: 0.3276 - val_accuracy: 0.8716
Epoch 2/10
71/71 - 0s - loss: 0.3127 - binary_crossentropy: 0.3126 - accuracy: 0.8727 - val_loss: 0.3095 - val_binary_crossentropy: 0.3094 - val_accuracy: 0.8709
Epoch 3/10
71/71 - 0s - loss: 0.3045 - binary_crossentropy: 0.3045 - accuracy: 0.8739 - val_loss: 0.3046 - val_binary_crossentropy: 0.3045 - val_accuracy: 0.8713
Epoch 4/10
71/71 - 0s - loss: 0.2944 - binary_crossentropy: 0.2944 - accuracy: 0.8773 - val_loss: 0.2958 - val_binary_crossentropy: 0.2957 - val_accuracy: 0.8700
Epoch 5/10
71/71 - 0s - loss: 0.2804 - binary_crossentropy: 0.2804 - accuracy: 0.8789 - val_loss: 0.2821 - val_binary_crossentropy: 0.2821 - val_accuracy: 0.8747
Epoch 6/10
71/71 - 0s - loss: 0.2655 - binary_crossentropy: 0.2654 - accuracy: 0.8824 - val_loss: 0.2799 - val_binary_crossentropy: 0.2799 - val_accuracy: 0.8740
Epoch 7/10
71/71 - 0s - loss

In [244]:
pred_ans = model.predict(test_model_input, batch_size=256)
# print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
# print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))

In [245]:
pred_ans

array([[0.05140117],
       [0.07697779],
       [0.00834781],
       ...,
       [0.34546733],
       [0.01153201],
       [0.03590134]], dtype=float32)

In [246]:
y_pred = [0 if x < 0.5 else 1 for x in pred_ans]

In [247]:
subscribe_map = {0: 'no', 1: 'yes'}

In [248]:
submission = pd.read_csv('./data/sample_submission.csv')
submission['subscribe'] = [subscribe_map[x] for x in y_pred]
submission.to_csv('./submission/submission_labelencoder_DeepFM.csv', index=None)

## targetencoser

In [36]:
train2 = train_data.copy()
test2 = test_data.copy()

In [37]:
train2['subscribe'] = train2['subscribe'].map({'no': 0, 'yes': 1})

In [38]:
cat_columns = test_data.select_dtypes(include='O').columns

In [39]:
for col in cat_columns:
    temp = train2.groupby(col)['subscribe'].agg(['mean', 'std', 'count']).reset_index()
    temp.columns = [col, col + '_mean', col + '_std', col + '_count']
    train2 = train2.merge(temp, on=col, how= 'left')
    test2 = test2.merge(temp, on=col, how= 'left')

In [40]:
train2.drop(columns=['RecordID'], inplace=True)
train2.drop(columns=cat_columns, inplace=True)

test2.drop(columns=['RecordID'], inplace=True)
test2.drop(columns=cat_columns, inplace=True)

In [41]:
train2['total_contact'] = train2['campaign'] + train2['previous']
train2['contact_diff'] = train2['campaign'] - train2['previous']

test2['total_contact'] = test2['campaign'] + test2['previous']
test2['contact_diff'] = test2['campaign'] - test2['previous']

In [42]:
datas2 = pd.concat([train2, test2], axis=0)

In [43]:
sparse_features = datas2.select_dtypes(include='O').columns
dense_features = datas2.select_dtypes(exclude='O').columns

In [44]:
dense_features

Index(['age', 'duration', 'campaign', 'pdays', 'previous', 'emp.var.rate',
       'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed',
       'subscribe', 'job_mean', 'job_std', 'job_count', 'marital_mean',
       'marital_std', 'marital_count', 'education_mean', 'education_std',
       'education_count', 'default_mean', 'default_std', 'default_count',
       'housing_mean', 'housing_std', 'housing_count', 'loan_mean', 'loan_std',
       'loan_count', 'contact_mean', 'contact_std', 'contact_count',
       'month_mean', 'month_std', 'month_count', 'day_of_week_mean',
       'day_of_week_std', 'day_of_week_count', 'poutcome_mean', 'poutcome_std',
       'poutcome_count'],
      dtype='object')

In [45]:
dense_features = dense_features.delete(dense_features.get_loc('subscribe'))

In [46]:
target = ['subscribe']

In [47]:
# 1.Label Encoding for sparse features,and do simple Transformation for dense features
for feat in sparse_features:
    lbe = LabelEncoder()
    datas2[feat] = lbe.fit_transform(datas2[feat])
    
mms = MinMaxScaler(feature_range=(0, 1))
datas2[dense_features] = mms.fit_transform(datas2[dense_features])

In [48]:
# 2.count #unique features for each sparse field,and record dense feature field name
fixlen_feature_columns = [SparseFeat(feat, vocabulary_size=datas2[feat].max() + 1, embedding_dim=4)
                          for i, feat in enumerate(sparse_features)] + [DenseFeat(feat, 1, ) for feat in dense_features]

In [49]:
#特征共用，放给低维linear_feature_columns和高维dnn_feature_columns
dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns

In [50]:
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

In [51]:
# 3.generate input data for model
# train, test = train_test_split(datas, test_size=0.2, random_state=2020)
# 数据集切分
train2 = datas2[datas2['subscribe'].notnull()]
test2 = datas2[datas2['subscribe'].isnull()]
test2.drop(['subscribe'], axis=1, inplace=True)

In [52]:
#增加检验，此种方式拆分的结果比使用deepctr.fit自带的拆分更稳定
train2_X,eval2_X,train2_y,eval2_y = train_test_split(train2[feature_names], train2[target], test_size=0.2, random_state=2022, stratify=train2[target])

In [53]:
train2_model_input = {name: train2_X[name] for name in feature_names}
eval2_model_input = {name: eval2_X[name] for name in feature_names}
test2_model_input = {name: test2[name] for name in feature_names}

In [54]:
# 4.Define Model,train,predict and evaluate
model = DeepFM(linear_feature_columns, dnn_feature_columns, task='binary')
model.compile("adam", "binary_crossentropy", metrics=['binary_crossentropy','accuracy'], )

In [55]:
# history = model.fit(train_model_input, train[target].values, batch_size=256, epochs=10, verbose=2, validation_split=0.2, )

history = model.fit(train2_model_input, train2_y.values, batch_size=256, epochs=10, verbose=2, 
                    validation_data=(eval2_model_input, eval2_y.values),)# validation_batch_size=50)

Epoch 1/10
71/71 - 2s - loss: 0.3867 - binary_crossentropy: 0.3867 - accuracy: 0.8428 - val_loss: 0.3272 - val_binary_crossentropy: 0.3272 - val_accuracy: 0.8656
Epoch 2/10
71/71 - 1s - loss: 0.3173 - binary_crossentropy: 0.3173 - accuracy: 0.8717 - val_loss: 0.3178 - val_binary_crossentropy: 0.3178 - val_accuracy: 0.8704
Epoch 3/10
71/71 - 1s - loss: 0.3116 - binary_crossentropy: 0.3116 - accuracy: 0.8748 - val_loss: 0.3124 - val_binary_crossentropy: 0.3123 - val_accuracy: 0.8738
Epoch 4/10
71/71 - 1s - loss: 0.3079 - binary_crossentropy: 0.3079 - accuracy: 0.8749 - val_loss: 0.3093 - val_binary_crossentropy: 0.3093 - val_accuracy: 0.8736
Epoch 5/10
71/71 - 1s - loss: 0.3021 - binary_crossentropy: 0.3021 - accuracy: 0.8770 - val_loss: 0.3068 - val_binary_crossentropy: 0.3068 - val_accuracy: 0.8724
Epoch 6/10
71/71 - 1s - loss: 0.2948 - binary_crossentropy: 0.2948 - accuracy: 0.8791 - val_loss: 0.3034 - val_binary_crossentropy: 0.3033 - val_accuracy: 0.8769
Epoch 7/10
71/71 - 1s - loss

In [56]:
pred_ans = model.predict(test2_model_input, batch_size=256)
# print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
# print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))

In [57]:
y_pred = [0 if x < 0.5 else 1 for x in pred_ans]

In [58]:
subscribe_map = {0: 'no', 1: 'yes'}
submission = pd.read_csv('./data/sample_submission.csv')
submission['subscribe'] = [subscribe_map[x] for x in y_pred]
submission.to_csv('./submission/submission_targetcoder_DeepFM.csv', index=None)

## onehot encoder

In [59]:
datas3 = pd.concat([train_data, test_data], axis=0)

In [60]:
datas3['subscribe'] = datas3['subscribe'].map({'no': 0, 'yes': 1})

In [61]:
datas3 = pd.get_dummies(datas3)

In [62]:
datas3

,RecordID,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,...,month_oct,month_sep,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success
0,13783,49,4457,1,115,2,1.4,92.479703,-35.498996,0.705058,...,0,0,0,1,0,0,0,1,0,0
1,23986,52,4797,1,402,2,-1.8,93.439161,-39.331320,4.245479,...,0,0,0,1,0,0,0,0,1,0
2,20663,46,169,1,999,1,-1.8,93.075000,-47.100000,1.445000,...,0,0,0,0,0,0,1,1,0,0
3,13958,26,376,28,999,0,1.4,93.444000,-36.100000,4.967000,...,0,0,1,0,0,0,0,0,1,0
4,28184,47,3033,1,252,4,-3.4,94.352376,-33.073620,1.208702,...,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7495,25105,50,3969,48,290,1,-1.8,94.077494,-39.985944,3.709514,...,0,0,1,0,0,0,0,1,0,0
7496,13297,35,596,8,424,3,1.4,93.302019,-49.737088,1.766907,...,0,0,0,0,0,0,1,1,0,0
7497,26780,53,107,3,999,0,-2.9,92.963000,-40.800000,1.281000,...,0,0,0,1,0,0,0,0,1,0
7498,16186,32,172,3,999,0,1.4,93.444000,-36.100000,4.963000,...,0,0,0,0,1,0,0,0,1,0


In [63]:
#去掉ID
datas3.drop(['RecordID'], axis=1, inplace=True)

In [64]:
datas3['total_contact'] = datas3['campaign'] + datas3['previous']
datas3['contact_diff'] = datas3['campaign'] - datas3['previous']

In [66]:
sparse_features = datas3.select_dtypes(include='O').columns
dense_features = datas3.select_dtypes(exclude='O').columns

In [70]:
dense_features

Index(['age', 'duration', 'campaign', 'pdays', 'previous', 'emp.var.rate',
       'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed',
       'subscribe', 'job_admin.', 'job_blue-collar', 'job_entrepreneur',
       'job_housemaid', 'job_management', 'job_retired', 'job_self-employed',
       'job_services', 'job_student', 'job_technician', 'job_unemployed',
       'job_unknown', 'marital_divorced', 'marital_married', 'marital_single',
       'marital_unknown', 'education_basic.4y', 'education_basic.6y',
       'education_basic.9y', 'education_high.school', 'education_illiterate',
       'education_professional.course', 'education_university.degree',
       'education_unknown', 'default_no', 'default_unknown', 'default_yes',
       'housing_no', 'housing_unknown', 'housing_yes', 'loan_no',
       'loan_unknown', 'loan_yes', 'contact_cellular', 'contact_telephone',
       'month_apr', 'month_aug', 'month_dec', 'month_jul', 'month_jun',
       'month_mar', 'month_may', 'month_no

In [71]:
dense_features = dense_features.delete(dense_features.get_loc('subscribe'))

In [72]:
target = ['subscribe']

In [74]:
# 1.Label Encoding for sparse features,and do simple Transformation for dense features
# for feat in sparse_features:
#     lbe = LabelEncoder()
#     datas[feat] = lbe.fit_transform(datas[feat])

mms = MinMaxScaler(feature_range=(0, 1))
datas3[dense_features] = mms.fit_transform(datas3[dense_features])

In [75]:
# 2.count #unique features for each sparse field,and record dense feature field name
fixlen_feature_columns = [SparseFeat(feat, vocabulary_size=datas[feat].max() + 1, embedding_dim=4)
                          for i, feat in enumerate(sparse_features)] + [DenseFeat(feat, 1, ) for feat in dense_features]

In [77]:
#特征共用，放给低维linear_feature_columns和高维dnn_feature_columns
dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns

In [78]:
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

In [81]:
# 3.generate input data for model
# train, test = train_test_split(datas, test_size=0.2, random_state=2020)
# 数据集切分
train3 = datas3[datas3['subscribe'].notnull()]
test3 = datas3[datas3['subscribe'].isnull()]
test3.drop(['subscribe'], axis=1, inplace=True)

In [83]:
#增加检验，此种方式拆分的结果比使用deepctr.fit自带的拆分更稳定
train3_X,eval3_X,train3_y,eval3_y = train_test_split(train3[feature_names], train3[target], test_size=0.2, random_state=2022, stratify=train3[target])

In [85]:
train3_model_input = {name: train3_X[name] for name in feature_names}
eval3_model_input = {name: eval3_X[name] for name in feature_names}
test3_model_input = {name: test3[name] for name in feature_names}

In [86]:
# 4.Define Model,train,predict and evaluate
model = DeepFM(linear_feature_columns, dnn_feature_columns, task='binary')
model.compile("adam", "binary_crossentropy", metrics=['binary_crossentropy','accuracy'], )

In [87]:
history = model.fit(train3_model_input, train3_y.values, batch_size=256, epochs=10, verbose=2, 
                    validation_data=(eval3_model_input, eval3_y.values),)# validation_batch_size=50)

Epoch 1/10
71/71 - 1s - loss: 0.3919 - binary_crossentropy: 0.3918 - accuracy: 0.8401 - val_loss: 0.3267 - val_binary_crossentropy: 0.3267 - val_accuracy: 0.8656
Epoch 2/10
71/71 - 0s - loss: 0.3170 - binary_crossentropy: 0.3170 - accuracy: 0.8703 - val_loss: 0.3171 - val_binary_crossentropy: 0.3171 - val_accuracy: 0.8673
Epoch 3/10
71/71 - 0s - loss: 0.3093 - binary_crossentropy: 0.3093 - accuracy: 0.8742 - val_loss: 0.3153 - val_binary_crossentropy: 0.3152 - val_accuracy: 0.8700
Epoch 4/10
71/71 - 0s - loss: 0.3028 - binary_crossentropy: 0.3027 - accuracy: 0.8774 - val_loss: 0.3120 - val_binary_crossentropy: 0.3120 - val_accuracy: 0.8702
Epoch 5/10
71/71 - 0s - loss: 0.2964 - binary_crossentropy: 0.2964 - accuracy: 0.8798 - val_loss: 0.3083 - val_binary_crossentropy: 0.3083 - val_accuracy: 0.8742
Epoch 6/10
71/71 - 0s - loss: 0.2889 - binary_crossentropy: 0.2889 - accuracy: 0.8844 - val_loss: 0.3081 - val_binary_crossentropy: 0.3080 - val_accuracy: 0.8758
Epoch 7/10
71/71 - 0s - loss

In [89]:
pred_ans = model.predict(test3_model_input, batch_size=256)
# print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
# print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))

In [90]:
pred_ans

array([[0.01469934],
       [0.0430117 ],
       [0.01442838],
       ...,
       [0.28019732],
       [0.01804689],
       [0.02290502]], dtype=float32)

In [91]:
y_pred = [0 if x < 0.5 else 1 for x in pred_ans]

In [92]:
subscribe_map = {0: 'no', 1: 'yes'}
submission = pd.read_csv('./data/sample_submission.csv')
submission['subscribe'] = [subscribe_map[x] for x in y_pred]
submission.to_csv('./submission/submission_onehot_DeepFM.csv', index=None)